In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns; sns.set()

sns.set_style("ticks")

In [ ]:
hydra_table = pd.read_csv("Fields-DR3/hydra_probgal0.7.csv", usecols=['RA', 'DEC', 'zml', 'r_petro', 'g_petro'])

In [ ]:
ra = hydra_table["RA"]
dec = hydra_table["DEC"]

In [ ]:
#Defining the center of Hydra (in degress) --> from NED 
ra_0 = 159.17
dec_0 = -27.524

def dist(ra, dec):
    return np.sqrt((ra - ra_0)**2 + (dec - dec_0)**2)

distances = dist(ra, dec)